In [ ]:
import sqlite3
import pandas as pd

# Function to connect to the SQLite database
def connect_to_db(db_name):
    try:
        conn = sqlite3.connect(db_name)
        print("Connection to database successful")
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

# Function to execute a SELECT query
def execute_select_query(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except sqlite3.Error as e:
        print(f"Error executing query: {e}")
        return None

# Function to execute an INSERT or UPDATE query
def execute_insert_update_query(conn, query, data):
    try:
        cursor = conn.cursor()
        cursor.execute(query, data)
        conn.commit()
        print("Query executed successfully")
    except sqlite3.Error as e:
        print(f"Error executing query: {e}")

# Function to read data from a text file and convert it to a CSV
def convert_txt_to_csv(txt_file, csv_file):
    df = pd.read_csv(txt_file, delimiter='\t')
    df.to_csv(csv_file, index=False)
    print(f"Data converted from {txt_file} to {csv_file}")

# Function to read grammatical forms from a CSV file into a dictionary
def read_grammatical_forms(csv_file):
    df = pd.read_csv(csv_file)
    forms = {
        'sgN': df['sgN'].values[0],
        'sgG': df['sgG'].values[0],
        'sgD': df['sgD'].values[0],
        'sgA': df['sgA'].values[0],
        'sgI': df['sgI'].values[0],
        'sgL': df['sgL'].values[0],
        'sgV': df['sgV'].values[0],
        'plN': df['plN'].values[0],
        'plG': df['plG'].values[0],
        'plD': df['plD'].values[0],
        'plA': df['plA'].values[0],
        'plI': df['plI'].values[0],
        'plL': df['plL'].values[0],
        'plV': df['plV'].values[0]
    }
    return forms

# Main execution
if __name__ == "__main__":
    # Connection parameters
    db_name = "pol_lab07.s3db"
    conn = connect_to_db(db_name)

    if conn:
        # First SQL query: Find and print any word in initial form (sgN)
        query1 = "SELECT sgN FROM words LIMIT 1"
        result1 = execute_select_query(conn, query1)
        print("First query result:", result1)

        # Second SQL query: Print all words starting with 'K' in case 'plG'
        query2 = "SELECT plG FROM words WHERE plG LIKE 'K%'"
        result2 = execute_select_query(conn, query2)
        print("Second query result:", result2)

        # Third SQL query: Add a new word 'przyjaciel' with gender '1'
        query3 = "INSERT INTO words (sgN, gender) VALUES (?, ?)"
        new_word = ('przyjaciel', 1)
        execute_insert_update_query(conn, query3, new_word)

        # Convert 'polish.txt' to 'polish.csv' and read grammatical forms
        txt_file = "polish.txt"
        csv_file = "polish.csv"
        convert_txt_to_csv(txt_file, csv_file)
        forms = read_grammatical_forms(csv_file)
        print("Grammatical forms:", forms)

        # Fourth SQL query: Update the new word's record with grammatical forms
        query4 = """
        UPDATE words
        SET sgG = ?, sgD = ?, sgA = ?, sgI = ?, sgL = ?, sgV = ?,
            plN = ?, plG = ?, plD = ?, plA = ?, plI = ?, plL = ?, plV = ?
        WHERE sgN = 'przyjaciel'
        """
        data = (forms['sgG'], forms['sgD'], forms['sgA'], forms['sgI'], forms['sgL'], forms['sgV'],
                forms['plN'], forms['plG'], forms['plD'], forms['plA'], forms['plI'], forms['plL'], forms['plV'])
        execute_insert_update_query(conn, query4, data)

        # Verify the update
        query5 = "SELECT * FROM words WHERE sgN = 'przyjaciel'"
        result5 = execute_select_query(conn, query5)
        print("Updated record:", result5)

        # Close the connection
        conn.close()
